In [7]:
import json
import os

In [8]:
triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
mappings_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/triples_v2/"
filtered_triples_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples/"

In [9]:
[elem for elem in os.listdir(triples_path) if ".triples" in elem]

['stexpanded.triples',
 'marvel.triples',
 'memoryalpha.triples',
 'swg.triples',
 'memorybeta.triples',
 'swtor.triples',
 'starwars.triples',
 'mcu.triples']

In [10]:
[elem for elem in os.listdir(mappings_path) if ".json" in elem]

['marvel_mapping.json',
 'swtor_mapping.json',
 'memoryalpha_mapping.json',
 'memorybeta_mapping.json',
 'stexpanded_mapping.json',
 'starwars_mapping.json',
 'mcu_mapping.json',
 'swg_mapping.json']

In [11]:
edges_to_keep = ["http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLink", "http://www.w3.org/1999/02/22-rdf-syntax-ns#type", "http://purl.org/dc/terms/subject", "http://www.w3.org/2004/02/skos/core#broader"]
edges_to_delete = ["http://dbkwik.webdatacommons.org/ontology/Image", "http://dbkwik.webdatacommons.org/ontology/wikiPageWikiLinkText"]
files_to_delete = "/File:"

In [12]:
triples = sorted([elem for elem in os.listdir(triples_path) if ".triples" in elem])
mappings = sorted([elem for elem in os.listdir(triples_path) if ".json" in elem])
num_of_files = len(triples)

for i in range(num_of_files):
    
    print("Processing file " + str(i+1) + " of " + str(num_of_files))
    
    triple = triples[i]
    mapping = mappings[i]
    # print("Triple file: " + triple)
    # print("Mapping file: " + mapping)
    
    edges_to_keep_num = []
    edges_to_delete_num = []
    
    kept = 0
    deleted = 0
    
    with open(triples_path + triple, "r") as t, open(mappings_path + mapping, "r") as m, open(filtered_triples_path + triple, "w") as f:
        
        data = json.load(m)
        for keep in edges_to_keep:
            edges_to_keep_num.append(str(data.get(keep)))
        for delete in edges_to_delete:
            edges_to_delete_num.append(str(data.get(delete)))
        # print(edges_to_keep_num)
        # print(edges_to_delete_num)
        filtered_data = {key: value for key, value in data.items() if files_to_delete in key}
        for key, value in filtered_data.items():
            edges_to_delete_num.append(str(value))
        # print(len(edges_to_delete_num))
        
        for line in t:
            triple_list = line.replace("\n", "").split("###")
            # print(triple_list)
            if any(edge in triple_list for edge in edges_to_keep_num) and not any(edge in triple_list for edge in edges_to_delete):
                f.write(line)
                kept += 1
            else:
                deleted += 1
        print(f"{triple}: {deleted}/{kept + deleted} triples are deleted")

Processing file 1 of 8
marvel.triples: 3544412/6733366 triples are deleted
Processing file 2 of 8
mcu.triples: 678855/1225115 triples are deleted
Processing file 3 of 8
memoryalpha.triples: 837481/2526928 triples are deleted
Processing file 4 of 8
memorybeta.triples: 829319/2542909 triples are deleted
Processing file 5 of 8
starwars.triples: 3457471/8246033 triples are deleted
Processing file 6 of 8
stexpanded.triples: 245270/567386 triples are deleted
Processing file 7 of 8
swg.triples: 127665/254354 triples are deleted
Processing file 8 of 8
swtor.triples: 71635/146148 triples are deleted
